In [272]:
import requests
from bs4 import BeautifulSoup
import re
from datetime import datetime
import pandas as pd
import schedule
import time


In [172]:
from pororo import Pororo
from flashtext import KeywordProcessor


In [183]:
#회사명 keyword화 시키기
def get_keyword(kdaq_path,kpi_path):
    keyword_processor = KeywordProcessor()
    kdaq=pd.read_excel(kdaq_path) 
    kpi=pd.read_excel(kpi_path)
    kpi_list=kpi["기업명"].to_list()
    kdaq_list=kdaq["기업명"].to_list()
    keyword_processor.add_keywords_from_list(kpi_list)
    keyword_processor.add_keywords_from_list(kdaq_list)
    return keyword_processor

In [293]:
#접속 
def get_articles(title,href,a_time):    
    common_url="https://news.naver.com/main/list.nhn?mode=LS2D&mid=sec&sid1=101&sid2=258&listType=title"
    date=str(datetime.today().strftime("%Y%m%d"))
    url=common_url+"&date="+date
    raw=requests.get(url, headers={"User-Agent":"Mozilla/5.0"})
    ht= BeautifulSoup(raw.text,"html.parser")
    articles=ht.select("ul.type02 > li")
    for art in articles:
        txt=art.a.text
        link=art.a.attrs["href"]
        link=link.replace("https://","")
        if txt in title:
            continue
        title.append(txt)
        href.append(link)
        now=datetime.now()
        now=now.strftime('%Y-%m-%d %H:%M:%S')
        a_time.append(now)
        
    return title,href,a_time

    
        
    

In [298]:
def return_result(review,title, href, a_time,keyword_processor):
    score=[]
    p_ti=[]
    p_li=[]
    p_sc=[]
    p_co=[]
    p_time=[]
    for i in range(len(title)):    
        sc=review(title[i])
        sc=sc-2.5
        score.append(sc)
        company=keyword_processor.extract_keywords(title[i])
        if len(company)>0:
            for j in range(len(company)):              
                p_ti.append(title[i])
                p_li.append(href[i])
                p_sc.append(sc)
                p_co.append(company[j])               
                
                p_time.append(a_time[i])
       
    p_dic={
        "company" : p_co,
        "title" : p_ti,
        "time" : p_time,
        "link" : p_li,
        "score" : p_sc   
        
    }
    p_df=pd.DataFrame(p_dic)
    p_df=p_df.sort_values(by=["score"],ascending=False)
    r_df=p_df.groupby(by=["company","title","time","link"]).sum()
    r_df=r_df.sort_values(by=["score"],ascending=False)
    return p_df, r_df

In [173]:
review = Pororo(task="review", lang="ko")

In [294]:
title=[]
href=[]
a_time=[]
kdaq_path="KOSDAQ_1468.xls"
kpi_path="KOSPI_800.xls"
keyword_processor=get_keyword(kdaq_path,kpi_path)
title, href,a_time=get_articles(title,href,a_time)

In [295]:
pp,result=return_result(review,title, href, a_time, keyword_processor)

In [299]:
title, href,a_time=get_articles(title,href,a_time)
pp,result=return_result(review,title, href, a_time, keyword_processor)


In [300]:
pp

,company,title,time,link,score
15,카카오,카카오뱅크 IPO 기대주 관심도 'TOP'…크래프톤은 호감도 1위,2021-02-22 13:37:56,news.naver.com/main/read.nhn?mode=LS2D&mid=sec...,2.20
7,씨젠,"씨젠, 삼성證 출신 박성우 부사장 영입…“M&A; 총괄”",2021-02-22 13:37:56,news.naver.com/main/read.nhn?mode=LS2D&mid=sec...,2.06
8,클래시스,"클래시스, 키메스2021 참가…""슈링크와 신제품 선보일 예정""",2021-02-22 13:37:56,news.naver.com/main/read.nhn?mode=LS2D&mid=sec...,2.02
4,나노,"[IPO] 나노씨엠에스 ""위조방지 특허기술로 시장 영향력 확대""",2021-02-22 13:37:56,news.naver.com/main/read.nhn?mode=LS2D&mid=sec...,2.01
12,소마젠,"소마젠, 마이크로바이옴 관련 특허 보유 세계 1위…""파이프라인 확대""",2021-02-22 13:37:56,news.naver.com/main/read.nhn?mode=LS2D&mid=sec...,2.00
16,대상,합병 대상 찾기도 전에 투자? 유망 스팩(SPAC) 고르는 법 [허란의 해외주식2.0],2021-02-22 13:37:56,news.naver.com/main/read.nhn?mode=LS2D&mid=sec...,1.95
18,MP한강,"MP한강, 中 최대 온라인 쇼핑몰 ‘티몰 글로벌’ 진입 임박",2021-02-22 13:37:56,news.naver.com/main/read.nhn?mode=LS2D&mid=sec...,1.93
22,에이치엘비,"반등하는 에이치엘비, 큰손들 담기 시작",2021-02-22 13:37:56,news.naver.com/main/read.nhn?mode=LS2D&mid=sec...,1.92
23,MP한강,"MP한강 ""中온라인 쇼핑몰 ‘티몰 글로벌’ 진입 임박..상반기 위생허가 취득 예상”",2021-02-22 13:37:56,news.naver.com/main/read.nhn?mode=LS2D&mid=sec...,1.90
14,제넥신,[시그널] 제넥신 관계사 네오이뮨텍 다음달 코스닥 입성,2021-02-22 13:37:56,news.naver.com/main/read.nhn?mode=LS2D&mid=sec...,1.84


In [316]:
while True:
    now=datetime.now()
    print (now)
    
    title, href,a_time=get_articles(title,href,a_time)
    pp,result=return_result(review,title, href, a_time, keyword_processor)
    rr=result.groupby(by=["company"]).sum().sort_values(by=["score"],ascending=False)
    
    print(result)
    print("============================================================================================================================================")
    print(rr)
    print("============================================================================================================================================")
    time.sleep(60)
    

2021-02-22 23:31:35.412476
                                                                                                                                score
company title                                           time                link                                                     
나노      [IPO출사표]나노씨엠에스 “나노 기술력으로 글로벌 기업 도약"             2021-02-22 16:03:05 news.naver.com/main/read.nhn?mode=LS2D&mid=sec&...   4.26
SK      [마켓인사이트][단독]SK그룹 SK종합화학 지분 매각한다…JV 설립 추진        2021-02-22 17:16:06 news.naver.com/main/read.nhn?mode=LS2D&mid=sec&...   3.24
        [마켓인사이트][단독]SK그룹 SK종합화학 매각한다…JV 설립 추진           2021-02-22 17:32:25 news.naver.com/main/read.nhn?mode=LS2D&mid=sec&...   3.14
SK텔레콤   혁신 또 혁신… SK텔레콤, AI·ESG 날개 달고 빅테크로 도약 [포춘클럽 라운지] 2021-02-22 18:00:54 news.naver.com/main/read.nhn?mode=LS2D&mid=sec&...   2.41
셀트리온    셀트리온 '매출 2조 클럽' 보인다                             2021-02-22 17:38:58 news.naver.com/main/read.nhn?mode=LS2D&mid=sec&...   2.30
...                                

KeyboardInterrupt: 

In [302]:
result

score
company title                                           time                link                                                     
나노      [IPO출사표]나노씨엠에스 “나노 기술력으로 글로벌 기업 도약"             2021-02-22 16:03:05 news.naver.com/main/read.nhn?mode=LS2D&mid=sec&...   4.26
SK      [마켓인사이트][단독]SK그룹 SK종합화학 지분 매각한다…JV 설립 추진        2021-02-22 17:16:06 news.naver.com/main/read.nhn?mode=LS2D&mid=sec&...   3.24
        [마켓인사이트][단독]SK그룹 SK종합화학 매각한다…JV 설립 추진           2021-02-22 17:32:25 news.naver.com/main/read.nhn?mode=LS2D&mid=sec&...   3.14
SK텔레콤   혁신 또 혁신… SK텔레콤, AI·ESG 날개 달고 빅테크로 도약 [포춘클럽 라운지] 2021-02-22 18:00:54 news.naver.com/main/read.nhn?mode=LS2D&mid=sec&...   2.41
셀트리온    셀트리온 '매출 2조 클럽' 보인다                             2021-02-22 17:38:58 news.naver.com/main/read.nhn?mode=LS2D&mid=sec&...   2.30
...                                                                                                                               ...
브이티지엠피  [마스크 쓴 상장사] 브이티지엠피·바이오플러스, 덴탈마스크 ‘허위광고’ 논란      2021-02-22 13:37:56 news.naver.com/main/read.nhn?mode=LS2D&mid=sec&...  -0.84
장원테크    장원테크, 작년 영업손실 188억…적자 381% 확대                   2021-02-22 17:51:01 news.naver.com/main/read.nhn?mode=LS2D&mid=sec&...  -0.84
한진칼     제동레저, 한진칼 지주 탈퇴                                 2021-02-22 17:44:26 news.naver.com/main/read.nhn?mode=LS2D&mid=sec&...  -1.15
대상      "주식 리딩방 속지마세요"…KB증권, 주린이 대상 교육 실시               2021-02-22 13:37:56 news.naver.com/main/read.nhn?mode=LS2D&mid=sec&...  -1.20
삼성전자    삼성전자만 잔뜩 담았다…넘쳐나는 '무늬만 ESG펀드'                   2021-02-22 17:46:37 news.naver.com/main/read.nhn?mode=LS2D&mid=sec&...  -1.23

[246 rows x 1 columns]

In [305]:
rr=result.groupby(by=["company"]).sum().sort_values(by=["score"],ascending=False)

In [307]:
rr.head(15)

,score
company,
셀트리온,14.80
바이오니아,11.00
SK,10.58
자안,10.44
나노,8.92
메디톡스,7.86
미래에셋대우,7.86
카카오,7.73
레이,6.80


In [315]:
pp[pp["company"]=="셀트리온"]

,company,title,time,link,score
202,셀트리온,셀트리온 '매출 2조 클럽' 보인다,2021-02-22 17:38:58,news.naver.com/main/read.nhn?mode=LS2D&mid=sec...,2.30
142,셀트리온,"셀트리온, 지난해 영업익 7121억…전년비 88.4%↑",2021-02-22 15:58:51,news.naver.com/main/read.nhn?mode=LS2D&mid=sec...,1.81
154,셀트리온,셀트리온 작년 매출액 1조8천491억원으로 64%↑…역대최대(종합),2021-02-22 16:16:57,news.naver.com/main/read.nhn?mode=LS2D&mid=sec...,1.66
139,셀트리온,"[속보]셀트리온 작년 영업이익 7,121억원···전년 대비 88.4%↑",2021-02-22 15:56:44,news.naver.com/main/read.nhn?mode=LS2D&mid=sec...,1.65
141,셀트리온,"셀트리온, 4분기 영업익 1647억…전년비 44.3%↑",2021-02-22 15:58:51,news.naver.com/main/read.nhn?mode=LS2D&mid=sec...,1.64
135,셀트리온,셀트리온 작년 영업이익 7천121억원…전년 대비 88.4%↑,2021-02-22 15:52:30,news.naver.com/main/read.nhn?mode=LS2D&mid=sec...,1.57
138,셀트리온,"셀트리온, 작년 4Q 영업익 1647억…전년比 44%↑",2021-02-22 15:55:40,news.naver.com/main/read.nhn?mode=LS2D&mid=sec...,1.56
140,셀트리온,"셀트리온, 2020년 4분기 영업익 1,647억 원...전년比 44.27% ↑",2021-02-22 15:56:44,news.naver.com/main/read.nhn?mode=LS2D&mid=sec...,1.54
20,셀트리온,"쥴릭파마, 셀트리온 코로나19 치료제 '렉키로나' 유통 개시",2021-02-22 13:37:56,news.naver.com/main/read.nhn?mode=LS2D&mid=sec...,1.07


In [312]:
result.index

MultiIndex([(    '나노',             '[IPO출사표]나노씨엠에스 “나노 기술력으로 글로벌 기업 도약"', ...),
            (    'SK',        '[마켓인사이트][단독]SK그룹 SK종합화학 지분 매각한다…JV 설립 추진', ...),
            (    'SK',           '[마켓인사이트][단독]SK그룹 SK종합화학 매각한다…JV 설립 추진', ...),
            ( 'SK텔레콤', '혁신 또 혁신… SK텔레콤, AI·ESG 날개 달고 빅테크로 도약 [포춘클럽 라운지]', ...),
            (  '셀트리온',                             '셀트리온 '매출 2조 클럽' 보인다', ...),
            (    'SK',        '[시그널] SK바이오사이언스, IPO 스타트···바이오팜 열풍도 넘어설까', ...),
            (  '메디톡스',                              '보톡스 전쟁 끝… 메디톡스 상한가', ...),
            (    '자안',                   '자안, 바이오헬스케어 연구소 인수…“특허 기술 확보”', ...),
            (  '대신증권',            '대신증권, 오익근號 출범 1년 만에 '호실적·주주가치 제고' 성공', ...),
            (   '카카오',            '카카오뱅크 IPO 기대주 관심도 'TOP'…크래프톤은 호감도 1위', ...),
            ...
            (  '제이웨이',            '[마스크 쓴 상장사] 제이웨이, 마스크 매출 6억에 재고는 24억', ...),
            (  '이아이디',                        '이아이디, 지난해 영업손 3.4억원…적자전환', ...),
            ( '클리노믹스',  